In [1]:
from __future__ import annotations

from pathlib import Path
WORK_DIR = Path.cwd()

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import smtplib

## Testing Alpha Vantage API

In [2]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='UX2OT39HH6HK14LB')
data, meta_data = ts.get_intraday('AAPL')

In [3]:
import numpy as np
import pandas as pd
import requests
import time

class AlphaVantageDataLoader():
    def __init__(
                 self, 
                 key: str):
        self.key=key

    
    def load_stock_data(self,
        tickers: list = [],
        frequency: str = 'daily', 
        output_size: str = 'full'
        ) -> dict: 

        freq_dict = {'intraday': 'TIME_SERIES_INTRADAY',
                     'daily': 'TIME_SERIES_DAILY',
                     'weekly': 'TIME_SERIES_WEEKLY',
                     'monthly': 'TIME_SERIES_MONTHLY'}
        
        if type(tickers) == str:
            tickers = [tickers]

        start = time.time()
        meta_dict = {} 
        df_dict = {}
        count=0
        
        for num, ticker in tqdm(enumerate(tickers)):
            link = f'https://www.alphavantage.co/query?function={freq_dict[frequency]}&symbol={ticker}&outputsize={output_size}&apikey={self.key}'
            stock = requests.get(link)
            keys_list = list(stock.json().keys())
            
            try:
                data, metadata = stock.json()[keys_list[-1]], stock.json()[keys_list[0]]

                df = pd.DataFrame(data).T
                df.columns = [col.split(' ')[-1] for col in df.columns]
                df = df.apply(pd.to_numeric)

                meta_dict[ticker] = meta_data
                df_dict[ticker]= df
                count+=1
            except:
                pass

            #Alpha Vantage only allows 5 API call per minute
            if (count+1)%5==0: 
                end = time.time()
                wait_time = (start+60) - end
                time.sleep(wait_time)
                start = time.time()
        
        self.meta = meta_dict

        return df_dict


In [4]:
ticker = pd.read_csv(WORK_DIR/'TSX.txt', sep='\t')

In [5]:
import random

ticker_list = ticker.loc[(~ticker.Symbol.str.contains('PR'))&
                           (~ticker.Description.str.contains('ETF'))&
                           (~ticker.Description.str.contains('Fund'))&
                           (~ticker.Description.str.contains('hdg'))]['Symbol'].tolist()

sampled_ticker = random.sample(ticker_list, 10)

In [6]:
# Initially wanted to use intraday data but intraday from alpha vantage doesn't seem to work for canadian stocks
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
stock_data = loader.load_stock_data(tickers=sampled_ticker)

10it [00:06,  1.65it/s]


In [7]:
sampled_ticker

['NEO.TO',
 'CJT.DB.D.TO',
 'AH.WT.TO',
 'MRT.UN.TO',
 'BSX.TO',
 'AR.TO',
 'PSK.TO',
 'GWO.TO',
 'SVM.TO',
 'LGT.A.TO']

In [8]:
stock_data.keys()

dict_keys(['NEO.TO'])

### Value Threshold Announcement`

In [17]:
# Initially wanted to use intraday data but intraday from alpha vantage doesn't seem to work for canadian stocks
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
stock_data = loader.load_stock_data(tickers=['BB.TO', 'CGX.TO'])

2it [00:06,  3.38s/it]


In [74]:
higher_threshold = {'BB.TO': 20, 'CGX.TO': 16.5, 'BDLP.TO': 20}
lower_threshold = {'BB.TO': 10, 'CGX.TO': 15, 'BDLP.TO': 30}
report_ticker = ['BB.TO', 'CGX.TO', 'L.TO', 'BDLP.TO']

download_list = list(set(list(higher_threshold.keys()) + list(lower_threshold.keys()) + report_ticker))

In [77]:
ticker = 'CGX.TO'
df1 = stock_data[ticker].reset_index()
df1['index'] = pd.to_datetime(df1['index'])
df1.set_index('index', inplace=True)

weekly_close = df1.resample('W')['close'].mean().tail(52)
current_close = weekly_close.iloc[-1]

In [78]:
arr = (weekly_close>weekly_close.shift(1)).astype(int).iloc[::-1].values

if np.any(arr==0):
    consecutive_week_high = np.where(arr==0)[0][0]
else:
    consecutive_week_high = np.where(arr==0)[0][0]

ma50 = stock_data[ticker].iloc[0:50]['close'].mean()
ma200 = stock_data[ticker].iloc[0:200]['close'].mean()

In [95]:
str1 = f"{ticker}\nMA50: {ma50:2f}\nMA200: {ma200:2f}\nConsecutive Weeks High: {consecutive_week_high}"
print("\n\n".join([str1]))

CGX.TO
MA50: 100000.000000
MA200: 10.468800
Consecutive Weeks High: 4


In [96]:
stock_data[ticker]

,open,high,low,close,volume
2021-06-08,16.30,16.65,16.11,16.40,1348641
2021-06-07,16.10,16.67,16.05,16.15,1248500
2021-06-04,15.85,16.08,15.61,15.96,812400
2021-06-03,15.98,16.40,15.58,15.89,1451700
2021-06-02,16.00,16.00,15.47,15.91,1151700
...,...,...,...,...,...
2005-01-10,14.81,15.20,14.80,14.90,212194
2005-01-07,14.76,14.99,14.76,14.86,23813
2005-01-06,14.75,14.95,14.75,14.75,29786
2005-01-05,14.70,14.76,14.70,14.75,94650


In [94]:
lower_report_list = [] 
for key, value in lower_threshold.items():
    

CGX.TO
MA50: 13.376000
MA200: 10.468800
Consecutive Weeks High: 4

CGX.TO
MA50: 13.376000
MA200: 10.468800
Consecutive Weeks High: 4


#### Sending Notification Email

In [ ]:
sender = "tuateststock@outlook.com" 
receiver = "wongsangaroon@gmail.com"
password = ("ImTestingStocks123") 
message = f'From: {sender}\nTo: {receiver}\nSubject: Stock Alert!!\n\nI Made It WORKKK!!\n'

In [ ]:

server = smtplib.SMTP('smtp-mail.outlook.com', 587)
server.starttls()
server.login(sender, password)
print("Login Success")
server.sendmail(sender, receiver, msg = message) 
print("Email was sent")
server.close()